In [1]:
import psycopg2
import pickle

In [2]:
with open('data/user_list.pickle', 'rb') as f:
    user_list = pickle.load(f)
# number of user: 147402, train dataset index 0 - 9999 (10,000), evaluation dataset index 20000 - 119999 (100,000)
eval_user_list = user_list[20000:120000].copy()
eval_user_list.sort()
print("user information loaded.")

user information loaded.


In [3]:
eval_user_list[:5]

['https://steamcommunity.com/id/----_-_-__-___---/',
 'https://steamcommunity.com/id/--Ulrich--/',
 'https://steamcommunity.com/id/--aldwin--/',
 'https://steamcommunity.com/id/--ranXerox--/',
 'https://steamcommunity.com/id/-16a/']

In [4]:
postgres_conn_param = {
    'host': 'localhost',
    'port': '5432',
    'user': 'postgres',
    'password': 'Lyc199412',
    'database': 'steam'}
conn = psycopg2.connect(**postgres_conn_param)
cur = conn.cursor()
SQL = """
SELECT reviewer_name, game_name, recommended, time_played
FROM game_reviews
WHERE reviewer_name = ANY(%s)
ORDER BY reviewer_name ASC
"""
cur.execute(SQL, (eval_user_list,))
reviews = cur.fetchall()


In [12]:
reviews[90000:90005]

[('https://steamcommunity.com/id/Boy_314/', 'Melatonin', True, Decimal('4.1')),
 ('https://steamcommunity.com/id/Boy_314/',
  'Dorfromantik',
  True,
  Decimal('113.1')),
 ('https://steamcommunity.com/id/Boy_314/',
  'Tabletop Simulator',
  True,
  Decimal('381.1')),
 ('https://steamcommunity.com/id/Boy_314/',
  'Patricks Parabox',
  True,
  Decimal('12.9')),
 ('https://steamcommunity.com/id/Boy_314/',
  'Spelunky 2',
  False,
  Decimal('23.2'))]

In [6]:
len(reviews)

2187495

In [10]:
eval_reviews = dict()

for u in eval_user_list:
    eval_reviews[u] = []

for reviewer_name, game_name, recommended, play_time in reviews:
    eval_reviews[reviewer_name].append((game_name, recommended, float(play_time)))
    

In [18]:
with open('data/eval_reviews.pickle', 'wb') as f:
    pickle.dump(eval_reviews, f)